In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import cv2

In [2]:
path = 'D:\\ILBS_14_8_21_Sanjeev\\Work_to_be_Completed_ILBS\\'

In [3]:
pid = []
image_file = []
for fname in os.listdir(path):
    if fname.isdigit():
        pid.append(fname)
        for fname_ins in os.listdir(path+fname):
            image_file.append(path+fname+'\\'+fname_ins)

In [4]:
df_f = pd.DataFrame(image_file)

In [5]:
df_f[0][1].split('\\')[-2]

'00910'

In [6]:
df_label = pd.read_excel(path+'\\'+'HVPG value with CT images.xlsx')

In [7]:
df_label.head()

,ID,Date,HVPG
0,9109,2019-01-30,21
1,19699,2019-02-13,12
2,31322,2019-02-15,17
3,45341,2019-02-13,16
4,49565,2019-02-06,10


In [8]:
labels = []
for row in df_f.values.tolist():
    id_ = int(row[0].split('\\')[-2])
    if id_ == 910:
        id_ = 9109
    for rows in df_label.values.tolist():
        if id_==rows[0]:
            labels.append([row[0],rows[-1]])
           # print(label)
        #break
    #print(row,id_)
    #break

In [9]:
len(labels)

35011

In [10]:
len(df_f)

35011

In [11]:
df = pd.DataFrame(labels,columns=['path','label'])

In [12]:
df.head()

,path,label
0,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
1,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
2,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
3,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21
4,D:\ILBS_14_8_21_Sanjeev\Work_to_be_Completed_I...,21


In [13]:

image_path = df['path'][0]
def imread(path):
    image_path = path
    IMG_HEIGHT = IMG_WIDTH =250
    image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255
    return image

In [14]:
df['imarray'] = df['path'].apply(imread)

In [14]:
from tensorflow import keras
#X = df['imarray'].values
y = np.array(df['label'])
y = keras.utils.to_categorical(y)

In [45]:
#sklearn image read
from skimage import io
all_images = []
IMG_HEIGHT = IMG_WIDTH =150
for row in df['path']:
    image= cv2.imread( row, cv2.IMREAD_GRAYSCALE)
    image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
    image=np.array(image)
    image = image.astype('float32')
    image /= 255
    image = np.expand_dims(image, axis=0)
    all_images.append(image)

In [46]:
train_ = np.array(all_images)

MemoryError: Unable to allocate 2.93 GiB for an array with shape (35011, 1, 150, 150) and data type float32

In [34]:
train_.shape

(35011, 150, 150)

In [36]:
X_final = np.empty((35011,150,150))
for i in range(35011):
    X_final[i] = train_[i]

In [43]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y,test_size=0.1)


MemoryError: Unable to allocate 601. MiB for an array with shape (3502, 150, 150) and data type float64

In [19]:
X_train.shape

(24507, 150, 150)

In [30]:
X_train_t = np.expand_dims(X_train,axis=4)
X_test_t = np.expand_dims(X_test, axis=4)

AxisError: axis 4 is out of bounds for array of dimension 4

In [27]:
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization

In [28]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', 
                 input_shape=[150, 150,1]))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(22, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=100, epochs=10, verbose=1)

ValueError: Error when checking input: expected conv2d_7_input to have 4 dimensions, but got array with shape (24507, 150, 150)